<a href="https://colab.research.google.com/github/abs-git/NLP/blob/main/English_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# glue data load

path = '/content/gdrive/MyDrive/Colab Notebooks/NLP'

with open(path + '/glue_train.txt', 'r') as f:
  train_sentences = f.readlines()

for i, sen in enumerate(train_sentences):
  train_sentences[i] = sen.rstrip("\n")

with open(path + '/glue_test.txt', 'r') as f:
  test_sentences = f.readlines()

for i, sen in enumerate(test_sentences):
  test_sentences[i] = sen.rstrip("\n")

print("train : {}".format(len(train_sentences)))
print("test : {}".format(len(test_sentences)))

print(train_sentences[:2])
print(test_sentences[:2])



train : 8551
test : 1063
["Our friends won't buy this analysis, let alone the next one we propose.", "One more pseudo generalization and I'm giving up."]
['Bill whistled past the house.', 'The car honked its way down the road.']


## Frequency (count) based embedding

Reference : https://datascienceschool.net/03%20machine%20learning/03.01.03%20Scikit-Learn%EC%9D%98%20%EB%AC%B8%EC%84%9C%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EB%8A%A5.html

In [ ]:
# Bag of Word (BoW)
import numpy as np
from collections import defaultdict


tokens = []
for sen in train_sentences[:3]:
  
  words = sen.split()
  for w in words:
    w = w.lower()
    if w not in tokens:
      tokens.append(w)


vocab = defaultdict(int)
for i, t in enumerate(tokens):
  vocab[t] += i+1


def BoW(sentences_list, vocab):

  BoW = []
  for sen in sentences_list:
    sen_words = sen.split()
    
    temp = []
    for t, i in vocab.items():
      if t in sen_words:
        count = sen_words.count(t)
        temp.append(count)
      else:
        temp.append(0)

    BoW.append(temp)

  return BoW


BoW = BoW(train_sentences[:3], vocab)


print("sentences     : {}".format(train_sentences[:3]))
print("tokens        : {}".format(tokens))
print("vocab         : {}".format(dict(vocab)))
print("sen to vector : {}".format(np.array(BoW)))

sentences     : ["Our friends won't buy this analysis, let alone the next one we propose.", "One more pseudo generalization and I'm giving up.", "One more pseudo generalization or I'm giving up."]
tokens        : ['our', 'friends', "won't", 'buy', 'this', 'analysis,', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose.', 'more', 'pseudo', 'generalization', 'and', "i'm", 'giving', 'up.', 'or']
vocab         : {'our': 1, 'friends': 2, "won't": 3, 'buy': 4, 'this': 5, 'analysis,': 6, 'let': 7, 'alone': 8, 'the': 9, 'next': 10, 'one': 11, 'we': 12, 'propose.': 13, 'more': 14, 'pseudo': 15, 'generalization': 16, 'and': 17, "i'm": 18, 'giving': 19, 'up.': 20, 'or': 21}
sen to vector : [[0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1]]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(token_pattern = string)
vectorizer = CountVectorizer()

# CountVectorizer는 기본적으로 길이가 2이상인 문자만 토큰으로 인식한다.
words_freq = vectorizer.fit_transform(train_sentences[:3]).toarray()

vocab = vectorizer.vocabulary_

print("sentences       : {}".format(np.array(train_sentences[:3])))
print("words frequency : {}".format(words_freq))    
print("vocab           : {}".format(vocab))


sentences       : ["Our friends won't buy this analysis, let alone the next one we propose."
 "One more pseudo generalization and I'm giving up."
 "One more pseudo generalization or I'm giving up."]
words frequency : [[1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1]
 [0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0]]
vocab           : {'our': 12, 'friends': 4, 'won': 19, 'buy': 3, 'this': 16, 'analysis': 1, 'let': 7, 'alone': 0, 'the': 15, 'next': 9, 'one': 10, 'we': 18, 'propose': 13, 'more': 8, 'pseudo': 14, 'generalization': 5, 'and': 2, 'giving': 6, 'up': 17, 'or': 11}


In [ ]:
# TF-IDF (Term Frequency-Inverse Document Frequency, 단어 빈도-역 문서 빈도)
from math import log

class tf_idf():
  def __init__(self, docs):
    self.docs = docs
    self.vocab = []
    self.nDocs = len(docs)

  def get_vocab(self):
    
    vocab = []
    for doc in self.docs:
      doc = doc.lower()
      words = doc.split(" ")

      for w in words:
        if w not in self.vocab:
          self.vocab.append(w)

    return self.vocab


  def get_tfidf(self):

    # df
    df = defaultdict(int)
    for doc in self.docs:
      doc = doc.lower()
      words = doc.split(" ")

      temp = []
      for t in self.vocab:
        if t in words:
          df[t] += 1


    # df -> idf
    idf = defaultdict(int)
    for t, df_value in df.items():
      idf[t] = log( self.nDocs / (df_value + 1) )



    tf_idf = []
    for doc in self.docs:
      doc = doc.lower()
      words = doc.split(" ")

      temp = []
      for t in self.vocab:
        if t in words:
          tf = words.count(t)              # tf
          result = round(tf * idf[t], 4)
          temp.append(result)

        else:
          temp.append(0)

      tf_idf.append(temp)

    
    return tf_idf


In [ ]:

vectorizer = tf_idf(train_sentences[:4])

vocab = vectorizer.get_vocab()
print(vocab)

matrix = vectorizer.get_tfidf()
print("result : {}".format(np.array(matrix)))

# nDocs = 4, df = 3인 경우 log 1 -> 0 이 되기 때문에 오류가 발생하기도 한다. 

['our', 'friends', "won't", 'buy', 'this', 'analysis,', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose.', 'more', 'pseudo', 'generalization', 'and', "i'm", 'giving', 'up.', 'or', 'study', 'verbs,', 'crazier', 'they', 'get.']
result : [[0.6931 0.6931 0.6931 0.6931 0.6931 0.6931 0.6931 0.6931 0.2877 0.6931
  0.     0.2877 0.6931 0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.2877 0.2877 0.6931 0.2877 0.2877 0.2877
  0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.2877 0.2877 0.     0.2877 0.2877 0.2877
  0.6931 0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.5754 0.
  0.     0.2877 0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.6931 0.6931 0.6931 0.6931 0.6931]]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(train_sentences[:4]).toarray()
vocab = vectorizer.vocabulary_

print("sentences : {}".format(train_sentences[0]))
print("TF-IDF    : {}".format(tfidf))
print("vocab     : {}".format(vocab))


sentences : Our friends won't buy this analysis, let alone the next one we propose.
TF-IDF    : [[0.29297188 0.29297188 0.         0.29297188 0.         0.29297188
  0.         0.         0.         0.29297188 0.         0.29297188
  0.18700015 0.         0.29297188 0.29297188 0.         0.
  0.23098239 0.         0.29297188 0.         0.         0.23098239
  0.29297188]
 [0.         0.         0.48217603 0.         0.         0.
  0.38015312 0.         0.38015312 0.         0.30776671 0.
  0.30776671 0.         0.         0.         0.38015312 0.
  0.         0.         0.         0.38015312 0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.38015312 0.         0.38015312 0.         0.30776671 0.
  0.30776671 0.48217603 0.         0.         0.38015312 0.
  0.         0.         0.         0.38015312 0.         0.
  0.        ]
 [0.         0.         0.         0.         0.34268741 0.
  0.         0.34268741 0.         0.         0.21873293 0

# Prediction based embedding

### Word2Vec

In [6]:
# CBOW (Continuous Bag Of Words) & Skip-gram
# Data processing
# CBOW와 Skip-gram 과정에서는 One hot encoding 을 수행하도록 한다.(torch의 경우 필수는 아님.)

import numpy as np


def create_vocab(sentences_list):
  
  vocab = []
  for sen in sentences_list:
    sen = sen.lower()
    tokens = sen.split(" ")

    for t in tokens:
      if t not in vocab:
        vocab.append(t)

  length = len(vocab)
  word_to_onehot = {}

  for i, v in enumerate(vocab):
    one_hot = np.zeros(length)
    one_hot[i] = 1
    word_to_onehot[v] = list(one_hot)
  
  return word_to_onehot


def create_word2vec_dataset(sentences_list, onehot_vocab, n = 2):     # n = window size

  contexts_data = []
  centers_data = []
  for sen in sentences_list:
    sen = sen.lower()
    tokens = sen.split(" ")                     # tokenizing

    for t_idx in range(n, len(tokens)-n):       # tokens[n:-n]
      center = tokens[t_idx]

      context = tokens[t_idx - n : t_idx+1 + n]
      context.remove(center)

      context_to_onehot = []
      for c in context:
        context_to_onehot.append(onehot_vocab[c])
        
      center_to_onehot = onehot_vocab[center]


      contexts_data.append(context_to_onehot)
      centers_data.append(center_to_onehot)

  return contexts_data, centers_data


def get_batch(contexts, centers, batch_size = 4):

  iter = len(centers) // batch_size

  batch = []
  idx = 0
  for i in range(iter):

    contexts_batch = np.array(contexts[idx : idx + batch_size])
    centers_batch = np.array(centers[idx : idx + batch_size])

    batch.append((contexts_batch, centers_batch))

    idx = i + batch_size

  return batch


nContext = 2
batch_size = 4

onehot_vocab = create_vocab(train_sentences)

contexts, centers = create_word2vec_dataset(train_sentences[:10], onehot_vocab, nContext)

word2vec_batch = get_batch(contexts, centers, batch_size)

print('contexts shape : {}'.format(np.array(contexts).shape))
print('centers shape : {}'.format(np.array(centers).shape))


contexts shape : (34, 4, 7845)
centers shape : (34, 7845)


In [ ]:
# torch CBOW

import torch
from torch import nn
from torch.optim import SGD
import torch.nn.functional as F


class CBOW(nn.Module):
  def __init__(self, vocab_size, embedding_size, context_size): 
    # 어휘 수 (onehot size), 임베딩 벡터 크기 (table size), 문맥 단어의 수 n (context number)
    
    super(CBOW, self).__init__()

    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.context_size = context_size

    self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)         # lookup table / 어휘 수, 뉴런 수
    self.linear1 = nn.Linear(2 * self.context_size * self.embedding_size, 512)
    self.linear2 = nn.Linear(512, self.vocab_size)


  def forward(self, inputs):   

    # inputs : (Batch size, Number of context words, Onehot context)
    # embedded : (Batch size, Number of context words, Onehot context, Onehot to embedding vector)
    # u : (Batch size, Number of context words, Onehot to embedding vector) 
    #   ->(Batch size, Number of context words * Onehot to embedding vector)
    # out : (Batch size, Onehot size embedding vector)

    batch_size = inputs.shape[0]
    
    embedded = self.embedding(inputs)

    u = embedded.sum(dim = 2)
    u = u.view(batch_size,-1)

    out = self.linear1(u)
    out = F.relu(out)

    out = self.linear2(out)
    out = F.log_softmax(out, dim = 1)

    return out


In [ ]:
# CBOW train

vocab_size = len(onehot_vocab)      # Number of words, 7845
embedding_size = 100                # Table size

losses = []

model = CBOW(vocab_size, embedding_size, nContext)

criterion = nn.CrossEntropyLoss()
optim = SGD(model.parameters(), lr=0.0001)

for epoch in range(10):
  
  batch_loss = 0
  for i, batch in enumerate(word2vec_batch):

    contexts = torch.tensor(batch[0], dtype = torch.long)
    centers = torch.tensor(batch[1], dtype = torch.float32)

    optim.zero_grad()
    probs = model(contexts)

    loss = criterion(probs, centers)
    loss.backward()
    optim.step()

    batch_loss = batch_loss + loss.item()

  print('epoch : {}, loss : {}'.format(epoch+1, batch_loss))
  
  losses.append(batch_loss)


epoch : 1, loss : 2.5208925254331023e+19
epoch : 2, loss : nan
epoch : 3, loss : nan
epoch : 4, loss : nan
epoch : 5, loss : nan
epoch : 6, loss : nan
epoch : 7, loss : nan
epoch : 8, loss : nan
epoch : 9, loss : nan
epoch : 10, loss : nan


In [ ]:
# torch Skip-gram

class SkipGram(nn.Module):
  def __init__(self, vocab_size, embedding_size, context_size): 
    # 어휘 수 (onehot size), 임베딩 벡터 크기 (table size), 문맥 단어의 수 n (context number)
    
    super(SkipGram, self).__init__()

    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.context_size = context_size

    self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)         # 룩업(lookup) : 테이블 / 어휘 수, 뉴런 수
    self.linear1 = nn.Linear(self.embedding_size, 512)
    self.linear2 = nn.Linear(512, 2 * self.context_size * self.vocab_size)


  def forward(self, inputs):   

    # inputs : (Batch size, Onehot center)
    # embedded : (Batch size, Onehot center, Onehot to embedding vector)
    # u : (Batch size, Onehot to embedding vector)
    # out : (Batch size, Onehot size embedding vector)
    #    -> (Batch size, Number of contexts, Onehot vector)

    batch_size = inputs.shape[0]
    
    embedded = self.embedding(inputs)

    u = embedded.sum(dim = 1)

    out = self.linear1(u)
    out = F.relu(out)

    out = self.linear2(out)
    out = F.log_softmax(out)

    out = out.view(batch_size, 2 * self.context_size, -1)

    return out



In [ ]:
# Skip gram train

vocab_size = len(onehot_vocab)      # 7845
embedding_size = 100

losses = []

model = SkipGram(vocab_size, embedding_size, nContext)

criterion = nn.CrossEntropyLoss()
optim = SGD(model.parameters(), lr=0.0001)

for epoch in range(10):
  total_loss = 0
  for i, batch in enumerate(word2vec_batch):
    contexts = torch.tensor(batch[0], dtype = torch.float32)
    centers = torch.tensor(batch[1], dtype = torch.long)

    optim.zero_grad()
    probs = model(centers)

    loss = criterion(probs, contexts)
    loss.backward()
    optim.step()

    batch_loss = batch_loss + loss.item()

  print('epoch : {}, loss : {}'.format(epoch+1, batch_loss))
  
  losses.append(batch_loss)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch : 1, loss : nan
epoch : 2, loss : nan
epoch : 3, loss : nan
epoch : 4, loss : nan
epoch : 5, loss : nan
epoch : 6, loss : nan
epoch : 7, loss : nan
epoch : 8, loss : nan
epoch : 9, loss : nan
epoch : 10, loss : nan


In [4]:
# SGNS (Skip-Gram with Negative Sampling)
# Data processing
# SGNS 에서는 Onehot encoding 없이 수행해본다.

import numpy as np
import random

def create_sgns_vocab(sentences_list):
  
  vocab = []
  for sen in sentences_list:
    sen = sen.lower()
    tokens = sen.split(" ")

    for t in tokens:
      if t not in vocab:
        vocab.append(t)

  length = len(vocab)
  word_to_label = {}

  init = 2                           # 0과 1은 pad, unk로 사용할 수 있도록 제외
  for i, v in enumerate(vocab):
    label = init + i
    word_to_label[v] = label
  
  return word_to_label


def create_ngns_dataset(sentences_list, vocab, nContext, nNega):

  center_data = []
  pos_data = []
  nega_data = []
  for sen in sentences_list:
    sen = sen.lower()
    tokens = sen.split(" ")                     # tokenizing

    for t_idx in range(nContext, len(tokens)-nContext):       # tokens[n:-n]
      
      vocab_words = list(vocab.keys())
      windows = tokens[t_idx - nContext : t_idx+1 + nContext]
      
      center_words = tokens[t_idx]
      pos_words = windows                                                 # context words
      pos_words.remove(center_words)
      nega_words = random.sample(set(vocab_words) - set(windows), nNega)  # negative sampling words

      center_data.append(vocab[center_words])
      
      temp_pos = []
      temp_nega = []
      for p, n in zip(pos_words, nega_words):
        temp_pos.append(vocab[p])
        temp_nega.append(vocab[n])

      pos_data.append(temp_pos)
      nega_data.append(temp_nega)

  return center_data, pos_data, nega_data


def get_batch(center_data, pos_data, nega_data, batch_size = 4):

  iter = len(center_data) // batch_size

  batch = []
  idx = 0
  for i in range(iter):

    center_batch = np.array(center_data[idx : idx + batch_size])
    pos_batch = np.array(pos_data[idx : idx + batch_size])
    nega_batch = np.array(nega_data[idx : idx + batch_size])

    batch.append((center_batch, pos_batch, nega_batch))

    idx = i + batch_size

  return batch


nContext = 2
nNega = 4
batch_size = 4

sgns_vocab = create_sgns_vocab(train_sentences[:3])

center_data, pos_data, nega_data = create_ngns_dataset(train_sentences[:3], sgns_vocab, nContext, nNega)

ngns_batch = get_batch(center_data, pos_data, nega_data, batch_size)


print(np.array(center_data).shape)
print(np.array(pos_data).shape)
print(np.array(nega_data).shape)

# print(center_data)
# print(pos_data)
# print(nega_data)


(17,)
(17, 4)
(17, 4)


In [25]:
# torch SGNS

import torch
from torch import nn
from torch.optim import SGD
import torch.nn.functional as F

class Embedding(nn.Module):

  def __init__(self, vocab_size, embedding_size, nContext): 
    super(Embedding, self).__init__()

    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.nContext = nContext
    
    self.in_embedding = nn.Embedding(vocab_size, embedding_size)
    self.out_embedding = nn.Embedding(vocab_size, embedding_size)
  
  def input_forward(self, input_word):
   
    return self.in_embedding(input_word)

  def output_forward(self, output_words):

    return self.out_embedding(output_words)


class SkipGramNegaLoss(nn.Module):

  def __init__(self): 
    super(SkipGramNegaLoss, self).__init__()

  def forward(self, input_vector, output_vectors, nega_vectors):

    batch_size, embedding_size = input_vector.shape

    input_vector = input_vector.view(batch_size, embedding_size, 1)    
    output_vectors = output_vectors.view(batch_size, 1, -1)
    nega_vectors = nega_vectors.view(batch_size, 1, -1)

    outputs_mul = torch.bmm(input_vector, output_vectors)               # bmm : batch matrix-matrix product 
    nega_mul = torch.bmm(input_vector, nega_vectors)

    outputs_loss = outputs_mul.sigmoid().log().mean(1)
    nega_loss = nega_mul.sigmoid().log().mean(1)

    loss = -(outputs_loss + nega_loss).mean()

    return loss



In [26]:
# SGNS train

vocab_size = len(onehot_vocab)
embedding_size = 100

losses = []

model = Embedding(vocab_size, embedding_size, nContext)

criterion = SkipGramNegaLoss()
optim = SGD(model.parameters(), lr=0.0001)

for epoch in range(10):
  batch_loss = 0
  for i, batch in enumerate(ngns_batch):
    centers = torch.tensor(batch[0], dtype = torch.long)
    positives = torch.tensor(batch[1], dtype = torch.long)
    negatives = torch.tensor(batch[2], dtype = torch.long)

    in_vector = model.input_forward(centers)
    out_vectors = model.output_forward(positives)
    nega_vectors = model.output_forward(negatives).neg()
    
    loss = criterion(in_vector, out_vectors, nega_vectors)

    optim.zero_grad()
    loss.backward()
    optim.step()

    batch_loss = batch_loss + loss.item()

  print('epoch : {}, loss : {}'.format(epoch+1, batch_loss))
  
  losses.append(batch_loss)



epoch : 1, loss : 6.339974999427795
epoch : 2, loss : 6.339974761009216
epoch : 3, loss : 6.3399741649627686
epoch : 4, loss : 6.3399739265441895
epoch : 5, loss : 6.3399738073349
epoch : 6, loss : 6.339973449707031
epoch : 7, loss : 6.339973211288452
epoch : 8, loss : 6.3399728536605835
epoch : 9, loss : 6.339972615242004
epoch : 10, loss : 6.339972376823425


In [29]:
from sklearn.metrics.pairwise import cosine_similarity

embeddings = model.in_embedding.weight.data.numpy()


In [54]:
# gensim built-in Word2Vec

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

tokens = []
for sen in train_sentences:
  words = re.sub(r"[^a-z0-9]+", " ", sen.lower())
  tokens.append(words)

corpus = []
corpus = [word_tokenize(sen) for sen in tokens]

print(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['our', 'friends', 'won', 't', 'buy', 'this', 'analysis', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose'], ['one', 'more', 'pseudo', 'generalization', 'and', 'i', 'm', 'giving', 'up'], ['one', 'more', 'pseudo', 'generalization', 'or', 'i', 'm', 'giving', 'up'], ['the', 'more', 'we', 'study', 'verbs', 'the', 'crazier', 'they', 'get'], ['day', 'by', 'day', 'the', 'facts', 'are', 'getting', 'murkier'], ['i', 'll', 'fix', 'you', 'a', 'drink'], ['fred', 'watered', 'the', 'plants', 'flat'], ['bill', 'coughed', 'his', 'way', 'out', 'of', 'the', 'restaurant'], ['we', 're', 'dancing', 'the', 'night', 'away'], ['herman', 'hammered', 'the', 'metal', 'flat'], ['the', 'critics', 'laughed', 'the', 'play', 'off', 'the', 'stage'], ['the', 'pond', 'froze', 'solid'], ['bill', 'rolled', 'out', 'of', 'the', 'room'], ['the', 'gardener', 'watered', 'the', 'flowers', 'flat'], ['the', 'gardener', 'watered', 'the', 'flowers'], ['bill', 'broke', 'the', 'bathtub', 'into', 'pieces'], ['bill', 'broke', 'th

In [57]:
# gensim built-in Word2Vec

from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, size = 100, window = 5, min_count = 1, workers = 4)

sims = model.wv.most_similar('our', topn = 10)

print(sims)

oov_word = model.wv.most_similar('ascbefg', topn = 10)  # not in vocabrary 에러 발생

[('his', 0.9999064207077026), ('an', 0.9998974800109863), ('and', 0.9998968839645386), ('s', 0.9998947381973267), ('her', 0.9998922348022461), ('herself', 0.9998897910118103), ('were', 0.9998884797096252), ('their', 0.9998874068260193), ('or', 0.9998868107795715), ('no', 0.9998849630355835)]


KeyError: ignored

### FastText

In [41]:

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

tokens = []
for sen in train_sentences:
  words = re.sub(r"[^a-z0-9]+", " ", sen.lower())
  tokens.append(words)

corpus = []
corpus = [word_tokenize(sen) for sen in tokens]

print(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['our', 'friends', 'won', 't', 'buy', 'this', 'analysis', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose'], ['one', 'more', 'pseudo', 'generalization', 'and', 'i', 'm', 'giving', 'up'], ['one', 'more', 'pseudo', 'generalization', 'or', 'i', 'm', 'giving', 'up'], ['the', 'more', 'we', 'study', 'verbs', 'the', 'crazier', 'they', 'get'], ['day', 'by', 'day', 'the', 'facts', 'are', 'getting', 'murkier'], ['i', 'll', 'fix', 'you', 'a', 'drink'], ['fred', 'watered', 'the', 'plants', 'flat'], ['bill', 'coughed', 'his', 'way', 'out', 'of', 'the', 'restaurant'], ['we', 're', 'dancing', 'the', 'night', 'away'], ['herman', 'hammered', 'the', 'metal', 'flat'], ['the', 'critics', 'laughed', 'the', 'play', 'off', 'the', 'stage'], ['the', 'pond', 'froze', 'solid'], ['bill', 'rolled', 'out', 'of', 'the', 'room'], ['the', 'gardener', 'watered', 'the', 'flowers', 'flat'], ['the', 'gardener', 'watered', 'the', 'flowers'], ['bill', 'broke', 'the', 'bathtub', 'into', 'pieces'], ['bill', 'broke', 'th

In [58]:
# FastText

# gensim의 모델들은 지정된 최소 워드 수 (min_count) 이상의 워드들에 대해서만 vocab를 생성하기 때문에
# corpus 내의 token의 수가 너무 적을 경우 학습을 할 수 없다. 

from gensim.models import FastText

model = FastText(sentences = corpus, size = 1000, window = 5, min_count = 1, workers = 4, sg = 1)

oov_word = model.most_similar('abcdefg')    # not in vocabrary 에러 발생 X

print(oov_word)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


[('leslie', 0.9999485611915588), ('sushi', 0.9999330043792725), ('leaks', 0.9999262690544128), ('announce', 0.9999244213104248), ('least', 0.999923825263977), ('dante', 0.9999207854270935), ('important', 0.9999205470085144), ('learn', 0.9999188184738159), ('seoul', 0.9999173879623413), ('held', 0.9999144673347473)]


# Frequency + Prediction based embedding

In [86]:
# GloVe

from nltk.tokenize import word_tokenize

def create_vocab(corpus):

  tokens = []
  sen_to_tokens = []
  for sen in corpus:
    sen = sen.lower()
    words = word_tokenize(sen)

    sen_to_tokens.append(words)
    tokens.extend(words)

  tokens = np.unique(tokens)

  vocab = {token: idx for idx, token in enumerate(tokens)}

  corpus_to_id = []
  for sen in sen_to_tokens:
    token_to_idx = []
    for token in sen:
      token_to_idx.append(vocab[token])
    corpus_to_id.append(token_to_idx)

  return vocab, corpus_to_id


def make_cooccur(vocab, corpus_to_id, window_size):

  vocab_size = len(vocab)

  co_mat = np.zeros((vocab_size, vocab_size))

  for sen in corpus_to_id:
    for center_i, center_token_id in enumerate(sen):

      left_i = center_i - 1
      right_i = center_i + 1

      if left_i >= 0:
        left_token_id = sen[left_i]
        co_mat[center_token_id, left_token_id] += 1

      if right_i < len(sen):
        right_token_id = sen[right_i]
        co_mat[center_token_id, right_token_id] += 1

  return co_mat


vocab, corpus_to_id = create_vocab(train_sentences[:3])
co_mat = make_cooccur(vocab, corpus_to_id, 2)

# print(vocab)
# print(corpus_to_id)

print(co_mat[:5])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0.]]


In [63]:
# GloVe package 

# !pip install glove

In [ ]:
from glove import Corpus, Glove

co_mat = Corpus()
co_mat.fit(train_sentences, window = 5)         # 동시 등장 행렬 (co-occurrence matrix) 생성

model = Glove(no_components = 100, learning_rate = 0.001)
model.fit(co_mat.matrix, epoch = 10, no_threads = 4)
model.add_dictionary(co_mat.dictionary)

test = model.most_similar("all")


# Language model based embedding

In [ ]:
# ELMo



